In [14]:
################### 买入后 上涨 大于30 的训练  
import numpy as np
import matplotlib.pyplot  as plt
import pandas as pd
import random

In [15]:
def sigmond(x):
    return 1/(1+np.exp(-x))

def reLu(x):
    return (np.abs(x)+x) / 2.0

def d_reLu(x):
    x[x<=0] = 0
    x[x>1] = 1
    return x

def softmax(x):
    x_row_max = x.max(axis=1).reshape(x.shape[0],1)
    x = x - x_row_max
    x_exp = np.exp(x)
    x_row_exp_sum = x_exp.sum(axis=1).reshape(x.shape[0],1)
    x_softmax = x_exp / x_row_exp_sum
    return x_softmax

def softmax_vector(x):
    x_exp = np.exp(x-x.max())
    return x_exp / x_exp.sum()

######### m 为样本个数 ###############
def J_sigmond(h,y,m):
    J = -y*np.log(h) - (1-y)*np.log(1-h)
    return J.sum() / m

In [16]:
###### 读取数据 ########
stock_data_org = pd.read_csv('stock_data_normal_u_20.csv',header=None)
stock_lable_org = pd.read_csv('stock_data_lable_normal_20.txt',header=None)

In [17]:
print(stock_data_org.shape[0])
print(stock_data_org.shape[0] / 6)
print(stock_lable_org.shape[0])

128094
21349.0
21349


In [18]:
################# 抽取出成功的标签，使成功与失败的比例是1:1.2 #####################
######### 所有的 股票数据，转换成（m,66 的形势）
all_x_data = stock_data_org.values
all_x_data = all_x_data.reshape(-1,66)
all_x_data = np.insert(all_x_data,0,1,axis=1)

################ 为 lable 加上 index
stock_lable_org['index'] = pd.DataFrame(np.arange(stock_lable_org.shape[0] -1))

############### 取出 所有 多空中 为 1 的数据 #####################
stock_lable_1 = stock_lable_org[stock_lable_org[2]==1]
############### 取出 所有 多空中 为 0 的数据 #####################
stock_lable_0 = stock_lable_org[stock_lable_org[2]==0] ##########
############### 对 多空为0 的数据进行抽样，数量为 多空为1 的1.2倍
stock_lable_0_sample = stock_lable_0.sample(n=int(stock_lable_1.shape[0]*1.2),replace=False)
############### 组合 0 的抽样数据与 1 的lable【所有数据】，形成 神经网络 的【训练集】与【测试集】
lable_nn_data = pd.concat([stock_lable_1,stock_lable_0_sample])
lable_nn_data = lable_nn_data.sort_values(by = 'index',ascending=True)
cut_idx = int(lable_nn_data.shape[0] * 0.7)
lable_nn_train = lable_nn_data.iloc[:cut_idx]
lable_nn_test = lable_nn_data.iloc[cut_idx:]
#print(lable_nn_test)
############### 根据 index 取出 所有对应的 股票数据 #####################
train_data_index = lable_nn_train['index'].values
#print(train_data_index)
test_data_index = lable_nn_test['index'].values
#print(test_data_index)
train_data_index = train_data_index.astype(int)
test_data_index = test_data_index.astype(int)
#print(test_data_index)
#print(test_data_index.tolist())
train_data = all_x_data[train_data_index.tolist()]
test_data = all_x_data[test_data_index.tolist()]
#train_data = np.insert(train_data,0,1,axis=1)
#test_data = np.insert(test_data,0,1,axis=1)
y_train = lable_nn_train[2].values.reshape(-1,1)
y_test = lable_nn_test[2].values.reshape(-1,1)

In [19]:
print(train_data.shape[0])
print(y_train.shape[0])
print(test_data.shape[0])
print(y_test.shape[0])

4168
4168
1787
1787


In [20]:
############## 验证数据是否取正确 ################
for i in range(10):
    random_index = random.randint(0,train_data_index.shape[0]-1)
    check_index = train_data_index[random_index]
    print('输入x',train_data[random_index])
    print('输入y',y_train[random_index])
    print('原有x',all_x_data[check_index])
    print('原有y',stock_lable_org.iloc[check_index])

输入x [ 1.      -1.03406 -1.41061 -0.91237 -1.10666 -0.73044  0.04526 -0.86923
 -1.19402  1.64806  1.37504  1.28343 -0.70169 -0.58489 -0.59956 -1.10666
  1.05725 -0.34366 -0.73015 -0.88366  0.68334  0.86646  0.86148 -0.4801
 -0.37846 -0.49529 -0.20121 -0.73746 -1.11799 -0.66061 -0.41812 -0.0402
  0.21662  0.22856 -0.14772  0.13762 -0.28675  0.20121  0.14468 -0.41899
 -0.13908  0.28019 -0.58285 -0.40498 -0.24614  0.73862  1.06656  1.79868
  1.00605  1.30528  1.85845  0.76492  0.90091 -0.76374 -0.8853  -0.72083
  1.62496  1.16977  0.49529  1.20727 -1.03931 -0.02307  1.63415  1.31472
 -0.94462 -1.16784 -1.4065 ]
输入y [0]
原有x [ 1.      -1.03406 -1.41061 -0.91237 -1.10666 -0.73044  0.04526 -0.86923
 -1.19402  1.64806  1.37504  1.28343 -0.70169 -0.58489 -0.59956 -1.10666
  1.05725 -0.34366 -0.73015 -0.88366  0.68334  0.86646  0.86148 -0.4801
 -0.37846 -0.49529 -0.20121 -0.73746 -1.11799 -0.66061 -0.41812 -0.0402
  0.21662  0.22856 -0.14772  0.13762 -0.28675  0.20121  0.14468 -0.41899
 -0.13908 

In [22]:
#################读取w1 与w2##################################
w1 = np.loadtxt("init_w1_20.txt",delimiter=',')
w2 = np.loadtxt("init_w2_20.txt",delimiter=',')
w2 = w2.reshape(-1,1)
#################根据文件内容，初始化w1与w2#####################

######### 设置学习率 
n = 0.5
######### 样本个数 ###########
m = train_data.shape[0] 
#print(w2[0:10,:])
print(w1[0:10,0:5])
print(w2[0:10])

[[ 0.68676001 -0.13673205 -0.80297671  0.10018054  0.90154739]
 [ 0.65911746 -0.35734716 -0.21062891  0.56935415  0.27758112]
 [ 0.54014901 -0.28419967 -0.67817668 -0.09486293 -0.81464902]
 [-0.47911248  0.54337487  0.69905965  0.41674459 -0.3883975 ]
 [ 0.79984069 -0.34090192  0.65996117 -0.32779722 -0.24276578]
 [ 0.96654396 -0.12179513 -0.70189875  0.57477777  0.53064581]
 [-0.28946137 -0.63242698  0.30962962 -0.61180937 -0.32434631]
 [ 0.70940736  0.64641348  0.95303495 -0.62714319 -0.47439158]
 [ 0.87879697 -0.79160336 -0.57475583 -0.29560801 -0.73659594]
 [-0.67463331 -0.25312786  0.72900536 -0.18837197  0.63740136]]
[[-0.40362757]
 [-0.8780669 ]
 [ 0.77245641]
 [ 0.74693006]
 [ 0.59254606]
 [ 0.52656013]
 [ 0.15518352]
 [-0.16709552]
 [ 0.05645342]
 [-0.44334487]]


In [26]:
for i in range(2000):

    ############ 隐藏层输入 ############
    hid_input = np.matmul(train_data,w1)  ##### 10000* 200
    ############ 隐藏层输出 ############
    hid_output = np.tanh(hid_input)
    ######## 为隐藏层增加偏执项 ############
    hid_output = np.insert(hid_output,0,1,axis=1)   # 10000* 201
    ############ 最后一层的输入 ############
    hpfs_input = np.matmul(hid_output,w2)
    ############ 最后一层的输出，即正向传播的输出值 ############
    #print(hpfs_input[0:10])
    hpfs_output = sigmond(hpfs_input)
    #print(hpfs_output[0:10])
    ############ 完成正向传播 ############

    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    ############ w2的权重误差计算 #######################

    d_cost_hyfs_input = hpfs_output - y_train
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / m
    #print(d_cost_w2.shape)
    #print(d_cost_w2[100:220,:])
    ######## d_cost_w2.shape = 393*10 ############
    #print(d_cost_hyfs_input.shape,w2.shape)
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T) 
    d_cost_hid_input = d_cost_hid_out*(1-hid_output*hid_output)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1)
    d_cost_w1 = np.matmul(train_data.T,d_cost_hid_input) / m
    print('----------cost is -------------',J_sigmond(hpfs_output,y_train,train_data.shape[0]),m)
    w1 = w1 - n*d_cost_w1
    w2 = w2 - n*d_cost_w2
    #print('w1 最大值位置',np.where(w1==w1.max()),',w1 最大值',np.max(w1))
    #print('d_w1 最大值位置',np.where(d_cost_w1==d_cost_w1.max()),',d_w1 最大值',np.max(d_cost_w1))
    np.savetxt("init_w1.txt",w1,delimiter=',')
    np.savetxt("init_w2.txt",w2,delimiter=',')

print('----------------done--------------------------')

----------cost is ------------- 0.16987730034655354 4168
----------cost is ------------- 0.16230742647553878 4168
----------cost is ------------- 0.1693744337959841 4168
----------cost is ------------- 0.16183622198456205 4168
----------cost is ------------- 0.16886993379921336 4168
----------cost is ------------- 0.16136360392325277 4168
----------cost is ------------- 0.1683637511470706 4168
----------cost is ------------- 0.16088952733826578 4168
----------cost is ------------- 0.16785583386815262 4168
----------cost is ------------- 0.16041394470915804 4168
----------cost is ------------- 0.16734612697947232 4168
----------cost is ------------- 0.15993680571421004 4168
----------cost is ------------- 0.1668345722069754 4168
----------cost is ------------- 0.15945805696769194 4168
----------cost is ------------- 0.16632110767160113 4168
----------cost is ------------- 0.15897764172444612 4168
----------cost is ------------- 0.16580566753587023 4168
----------cost is ------------- 0.

----------cost is ------------- 0.09021462314603053 4168
----------cost is ------------- 0.08879323277568496 4168
----------cost is ------------- 0.08867808317187305 4168
----------cost is ------------- 0.08743876446360047 4168
----------cost is ------------- 0.08732694753360892 4168
----------cost is ------------- 0.08624697664945047 4168
----------cost is ------------- 0.08614635300877053 4168
----------cost is ------------- 0.08520424720680125 4168
----------cost is ------------- 0.08511998394270615 4168
----------cost is ------------- 0.08429617599985954 4168
----------cost is ------------- 0.08423158810244963 4168
----------cost is ------------- 0.08350877039745293 4168
----------cost is ------------- 0.08346602060474696 4168
----------cost is ------------- 0.08282923778925413 4168
----------cost is ------------- 0.08280988660228619 4168
----------cost is ------------- 0.08224646065879969 4168
----------cost is ------------- 0.0822518861279587 4168
----------cost is ------------- 

----------cost is ------------- 0.06671354031259677 4168
----------cost is ------------- 0.06661627821092961 4168
----------cost is ------------- 0.06651958438635121 4168
----------cost is ------------- 0.06642344280500752 4168
----------cost is ------------- 0.06632783789297646 4168
----------cost is ------------- 0.0662327552081922 4168
----------cost is ------------- 0.06613818070549027 4168
----------cost is ------------- 0.06604410133518697 4168
----------cost is ------------- 0.06595050439049192 4168
----------cost is ------------- 0.06585737804370717 4168
----------cost is ------------- 0.0657647107636141 4168
----------cost is ------------- 0.0656724917983709 4168
----------cost is ------------- 0.06558071065261775 4168
----------cost is ------------- 0.06548935752462569 4168
----------cost is ------------- 0.0653984228345906 4168
----------cost is ------------- 0.06530789762234543 4168
----------cost is ------------- 0.06521777311971816 4168
----------cost is ------------- 0.0

----------cost is ------------- 0.055661846058991174 4168
----------cost is ------------- 0.05559782942896332 4168
----------cost is ------------- 0.05553393900754234 4168
----------cost is ------------- 0.055470174322585816 4168
----------cost is ------------- 0.055406534834409576 4168
----------cost is ------------- 0.05534302008056138 4168
----------cost is ------------- 0.05527962953927315 4168
----------cost is ------------- 0.055216362756528276 4168
----------cost is ------------- 0.05515321922997786 4168
----------cost is ------------- 0.055090198512555345 4168
----------cost is ------------- 0.05502730012326454 4168
----------cost is ------------- 0.0549645236202094 4168
----------cost is ------------- 0.054901868546199384 4168
----------cost is ------------- 0.05483933446235756 4168
----------cost is ------------- 0.054776920938380075 4168
----------cost is ------------- 0.054714627535788915 4168
----------cost is ------------- 0.054652453855009775 4168
----------cost is -----

----------cost is ------------- 0.0486623162581574 4168
----------cost is ------------- 0.04867776854051699 4168
----------cost is ------------- 0.048739541723685935 4168
----------cost is ------------- 0.04876553885287039 4168
----------cost is ------------- 0.048849021406691034 4168
----------cost is ------------- 0.048887768662218486 4168
----------cost is ------------- 0.04899824331672095 4168
----------cost is ------------- 0.049052659829987105 4168
----------cost is ------------- 0.04919716178381037 4168
----------cost is ------------- 0.04927121349643286 4168
----------cost is ------------- 0.049459318232476304 4168
----------cost is ------------- 0.04955855483424095 4168
----------cost is ------------- 0.04980363310457477 4168
----------cost is ------------- 0.049936087322257745 4168
----------cost is ------------- 0.05025738478835422 4168
----------cost is ------------- 0.050435146745030676 4168
----------cost is ------------- 0.050861394544506965 4168
----------cost is ------

...

----------cost is ------------- 0.03382211879775141 4168
----------cost is ------------- 0.0337916973578053 4168
----------cost is ------------- 0.03376132121862331 4168
----------cost is ------------- 0.033730990261553115 4168
----------cost is ------------- 0.03370070436857211 4168
----------cost is ------------- 0.03367046342228121 4168
----------cost is ------------- 0.03364026730589866 4168
----------cost is ------------- 0.03361011590325399 4168
----------cost is ------------- 0.03358000909878209 4168
----------cost is ------------- 0.033549946777517235 4168
----------cost is ------------- 0.033519928825087426 4168
----------cost is ------------- 0.033489955127708526 4168
----------cost is ------------- 0.03346002557217873 4168
----------cost is ------------- 0.03343014004587295 4168
----------cost is ------------- 0.033400298436737294 4168
----------cost is ------------- 0.03337050063328374 4168
----------cost is ------------- 0.03334074652458477 4168
----------cost is ---------

----------cost is ------------- 0.02983368875079048 4168
----------cost is ------------- 0.029808831783019785 4168
----------cost is ------------- 0.029784007427951478 4168
----------cost is ------------- 0.02975921562004228 4168
----------cost is ------------- 0.02973445629396211 4168
----------cost is ------------- 0.02970972938459306 4168
----------cost is ------------- 0.02968503482702816 4168
----------cost is ------------- 0.029660372556570287 4168
----------cost is ------------- 0.029635742508730946 4168
----------cost is ------------- 0.029611144619229274 4168
----------cost is ------------- 0.029586578823990826 4168
----------cost is ------------- 0.029562045059146492 4168
----------cost is ------------- 0.029537543261031456 4168
----------cost is ------------- 0.02951307336618405 4168
----------cost is ------------- 0.02948863531134472 4168
----------cost is ------------- 0.02946422903345496 4168
----------cost is ------------- 0.029439854469656263 4168
----------cost is ----

----------cost is ------------- 0.026561325446075954 4168
----------cost is ------------- 0.026540577903689615 4168
----------cost is ------------- 0.026519855303915575 4168
----------cost is ------------- 0.02649915760364068 4168
----------cost is ------------- 0.0264784847598676 4168
----------cost is ------------- 0.02645783672971435 4168
----------cost is ------------- 0.026437213470414035 4168
----------cost is ------------- 0.026416614939314437 4168
----------cost is ------------- 0.0263960410938776 4168
----------cost is ------------- 0.02637549189167951 4168
----------cost is ------------- 0.026354967290409707 4168
----------cost is ------------- 0.026334467247870953 4168
----------cost is ------------- 0.02631399172197881 4168
----------cost is ------------- 0.026293540670761374 4168
----------cost is ------------- 0.0262731140523588 4168
----------cost is ------------- 0.026252711825023055 4168
----------cost is ------------- 0.02623233394711751 4168
----------cost is -------

----------cost is ------------- 0.023776228848635388 4168
----------cost is ------------- 0.023758719513563592 4168
----------cost is ------------- 0.02374122989272098 4168


KeyboardInterrupt: 

In [27]:
################# 先用 验证集进行验证 ##################
test_error_count = 0
#print(test_data.shape[0])
for i in range(test_data.shape[0]-1):
    x_test_data = test_data[i]
    hid_input = np.matmul(x_test_data,w1)
    hid_output = np.tanh(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=0)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = sigmond(hpfs_input)
    #print(y_test.shape)
    if (hpfs_output>0.5 and y_test[i] ==0 ) or (hpfs_output <0.5 and y_test[i] ==1):
        test_error_count = test_error_count +1
        print(hpfs_output,y_test[i])
print(test_error_count/test_data.shape[0])

[0.19276912] [1]
[0.74620382] [0]
[0.57714494] [0]
[0.80796573] [0]
[0.99992641] [0]
[0.99919674] [0]
[0.9996768] [0]
[0.0988937] [1]
[0.95107209] [0]
[0.74136265] [0]
[0.99999709] [0]
[0.00096929] [1]
[0.04691654] [1]
[0.76074963] [0]
[0.07241529] [1]
[0.58050282] [0]
[0.00602109] [1]
[0.8509904] [0]
[0.8965243] [0]
[0.02798016] [1]
[0.07700021] [1]
[0.99905668] [0]
[0.99738434] [0]
[0.86576951] [0]
[0.37134889] [1]
[0.00731727] [1]
[8.82931258e-05] [1]
[0.00340874] [1]
[0.84691291] [0]
[0.97419517] [0]
[0.06505023] [1]
[0.9209693] [0]
[0.83909923] [0]
[0.8423273] [0]
[0.68152686] [0]
[0.98075126] [0]
[0.00018244] [1]
[0.96928252] [0]
[0.86906935] [0]
[0.13269117] [1]
[0.20649634] [1]
[0.16216226] [1]
[0.69363576] [0]
[0.01772947] [1]
[0.09019865] [1]
[0.00977611] [1]
[0.29862196] [1]
[0.05788979] [1]
[0.88744083] [0]
[0.5026505] [0]
[0.84694199] [0]
[0.87586677] [0]
[0.53980058] [0]
[0.30112669] [1]
[0.8846639] [0]
[0.89252003] [0]
[0.96764687] [0]
[0.90606753] [0]
[0.49768555] [1]
[

In [28]:
################# 用所有数据验证 ##################
all_error_count = 0
#all_x_data = np.insert(all_x_data,0,1,axis=1)
#print(test_data.shape[0])
for i in range(all_x_data.shape[0]-1):
    x_test_data = all_x_data[i]
    hid_input = np.matmul(x_test_data,w1)
    hid_output = np.tanh(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=0)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = sigmond(hpfs_input)
    #print(y_test.shape)
    if (hpfs_output>0.5 and stock_lable_org.iloc[i,2] ==0 ) or (hpfs_output <0.5 and stock_lable_org.iloc[i,2] ==1):
        all_error_count = all_error_count +1
        print(hpfs_output,stock_lable_org.iloc[i,2])
print(all_error_count/all_x_data.shape[0])

[0.91699877] 0
[0.99675446] 0
[0.95488049] 0
[0.99841512] 0
[0.98281701] 0
[0.93102647] 0
[0.94095508] 0
[0.99999859] 0
[0.97633082] 0
[0.74374169] 0
[0.94507206] 0
[0.99976807] 0
[0.77774098] 0
[0.99980569] 0
[0.97892927] 0
[0.77423389] 0
[0.89189862] 0
[0.99979091] 0
[0.99999755] 0
[0.98169095] 0
[0.74280901] 0
[0.99559027] 0
[0.98922081] 0
[0.99852474] 0
[0.9692594] 0
[0.99998598] 0
[0.99628793] 0
[0.95603273] 0
[0.95974159] 0
[0.83691429] 0
[0.81062812] 0
[0.7898574] 0
[0.76791359] 0
[0.99921705] 0
[0.99996228] 0
[0.91557735] 0
[0.97764781] 0
[0.96821688] 0
[0.70153224] 0
[0.9346923] 0
[0.99981206] 0
[0.99631447] 0
[0.90430629] 0
[0.55935093] 0
[0.81000143] 0
[0.95893307] 0
[0.99995494] 0
[0.99803204] 0
[0.93567799] 0
[0.62601282] 0
[0.99845822] 0
[0.97947705] 0
[0.65509152] 0
[0.99996951] 0
[0.89878696] 0
[0.99980741] 0
[0.75833564] 0
[0.99626672] 0
[0.62006624] 0
[0.97058045] 0
[0.99996631] 0
[0.99701844] 0
[0.93329713] 0
[0.86907195] 0
[0.76663257] 0
[0.99676785] 0
[0.92066121] 

[0.98698631] 0
[0.63903912] 0
[0.98823383] 0
[0.80334147] 0
[0.99999431] 0
[0.51524139] 0
[0.95240786] 0
[0.98787038] 0
[0.999833] 0
[0.98480402] 0
[0.72527624] 0
[0.95647961] 0
[0.5069821] 0
[0.99968637] 0
[0.91900421] 0
[0.99786233] 0
[0.94543928] 0
[0.99282916] 0
[0.9997712] 0
[0.97820506] 0
[0.78725378] 0
[0.90314648] 0
[0.99999893] 0
[0.82395822] 0
[0.79014294] 0
[0.9901373] 0
[0.99999951] 0
[0.75465037] 0
[0.99771572] 0
[0.8814127] 0
[0.65734624] 0
[0.6903517] 0
[0.58245315] 0
[0.99744728] 0
[0.96896866] 0
[0.93771416] 0
[0.9240119] 0
[0.80549589] 0
[0.82664091] 0
[0.96609713] 0
[0.50519082] 0
[0.62385199] 0
[0.97501188] 0
[0.99585172] 0
[0.99715042] 0
[0.71371607] 0
[0.79080313] 0
[0.89163984] 0
[0.99971112] 0
[0.74240357] 0
[0.99324043] 0
[0.78783595] 0
[0.99329998] 0
[0.99952349] 0
[0.99998604] 0
[0.98466171] 0
[0.90198822] 0
[0.99923856] 0
[0.92652545] 0
[0.88379993] 0
[0.9895695] 0
[0.99146664] 0
[0.94367887] 0
[0.74144828] 0
[0.65159638] 0
[0.99981347] 0
[0.5366738] 0
[0.99

[0.99977716] 0
[0.57272629] 0
[0.99547881] 0
[0.79849639] 0
[0.92889523] 0
[0.75514796] 0
[0.83871547] 0
[0.99998436] 0
[0.71838424] 0
[0.96853893] 0
[0.96459941] 0
[0.99465442] 0
[0.97772763] 0
[0.99880168] 0
[0.58354049] 0
[0.99731698] 0
[0.76423665] 0
[0.99299493] 0
[0.95750632] 0
[0.87943747] 0
[0.9975725] 0
[0.96668573] 0
[0.84531227] 0
[0.99814488] 0
[0.80991217] 0
[0.91774124] 0
[0.96630674] 0
[0.74234563] 0
[0.98665139] 0
[0.99884526] 0
[0.50516464] 0
[0.92159609] 0
[0.99878393] 0
[0.99301535] 0
[0.94589106] 0
[0.98741049] 0
[0.99993503] 0
[0.90060877] 0
[0.93952701] 0
[0.60616231] 0
[0.9996629] 0
[0.99944644] 0
[0.57578792] 0
[0.92282458] 0
[0.69692458] 0
[0.73851065] 0
[0.9746145] 0
[0.98110582] 0
[0.52155219] 0
[0.99536287] 0
[0.86202181] 0
[0.62936045] 0
[0.82098883] 0
[0.81971032] 0
[0.97523872] 0
[0.98774277] 0
[0.92769588] 0
[0.99313421] 0
[0.99931984] 0
[0.98864504] 0
[0.99996822] 0
[0.99195655] 0
[0.99331388] 0
[0.99422509] 0
[0.99836028] 0
[0.99128716] 0
[0.97751035] 

[0.99998416] 0
[0.99593413] 0
[0.99999702] 0
[0.63099479] 0
[0.99964537] 0
[0.80868941] 0
[0.99955746] 0
[0.99176533] 0
[0.93529071] 0
[0.90189121] 0
[0.99992608] 0
[0.92497687] 0
[0.99645627] 0
[0.80276615] 0
[0.75018065] 0
[0.62824443] 0
[0.99999574] 0
[0.99152043] 0
[0.99951738] 0
[0.99840132] 0
[0.95930558] 0
[0.98305108] 0
[0.97483067] 0
[0.98175284] 0
[0.96429913] 0
[0.77144887] 0
[0.99825838] 0
[0.94880851] 0
[0.99553613] 0
[0.8218097] 0
[0.74542435] 0
[0.97465018] 0
[0.99676479] 0
[0.99996803] 0
[0.99996502] 0
[0.95347489] 0
[0.6724361] 0
[0.98882809] 0
[0.65609717] 0
[0.98915726] 0
[0.99994804] 0
[0.98630294] 0
[0.99966678] 0
[0.79133279] 0
[0.96868745] 0
[0.99865577] 0
[0.96492359] 0
[0.99994498] 0
[0.94618099] 0
[0.91072806] 0
[0.9984885] 0
[0.99753918] 0
[0.63911074] 0
[0.96937339] 0
[0.93103218] 0
[0.72126331] 0
[0.92407792] 0
[0.95945179] 0
[0.99894139] 0
[0.99634026] 0
[0.95081007] 0
[0.98021389] 0
[0.87919664] 0
[0.9750986] 0
[0.96866679] 0
[0.99999856] 0
[0.93255227] 0

[0.88386009] 0
[0.84153084] 0
[0.77210587] 0
[0.95616778] 0
[0.77004776] 0
[0.99818334] 0
[0.99999946] 0
[0.88300543] 0
[0.69705816] 0
[0.86160123] 0
[0.99829897] 0
[0.90892119] 0
[0.91072774] 0
[0.93603392] 0
[0.93467007] 0
[0.88104478] 0
[0.99677556] 0
[0.83668873] 0
[0.98183354] 0
[0.8299557] 0
[0.99796044] 0
[0.76338366] 0
[0.91311751] 0
[0.70021909] 0
[0.99951583] 0
[0.71618172] 0
[0.99970207] 0
[0.9454304] 0
[0.72010938] 0
[0.52635842] 0
[0.87439495] 0
[0.98128035] 0
[0.99470318] 0
[0.8849184] 0
[0.98971391] 0
[0.99995149] 0
[0.72899655] 0
[0.99554011] 0
[0.7210323] 0
[0.93512327] 0
[0.99446881] 0
[0.93329913] 0
[0.99772942] 0
[0.65719056] 0
[0.99999924] 0
[0.87477226] 0
[0.94054672] 0
[0.98003689] 0
[0.56782471] 0
[0.66138811] 0
[0.79904439] 0
[0.65957291] 0
[0.78892624] 0
[0.87003593] 0
[0.80681217] 0
[0.7951869] 0
[0.9974209] 0
[0.82370022] 0
[0.99645708] 0
[0.96186573] 0
[0.95567182] 0
[0.85270002] 0
[0.90581522] 0
[0.94078436] 0
[0.98965284] 0
[0.98353875] 0
[0.76599311] 0
[

[0.99952316] 0
[0.79969116] 0
[0.84095212] 0
[0.96506662] 0
[0.62634044] 0
[0.99785236] 0
[0.9974226] 0
[0.92417924] 0
[0.77116847] 0
[0.85830195] 0
[0.01579212] 1
[0.99997132] 0
[0.90783275] 0
[0.01268294] 1
[0.00190723] 1
[0.89696489] 0
[0.55691518] 0
[0.96908336] 0
[0.88733227] 0
[0.97538122] 0
[0.57799959] 0
[0.99325964] 0
[0.90695774] 0
[0.14135382] 1
[0.99996002] 0
[0.9337488] 0
[0.99940765] 0
[0.8430248] 0
[0.13407136] 1
[0.75743399] 0
[0.00023379] 1
[0.99999698] 0
[0.90646853] 0
[0.99315502] 0
[0.86019111] 0
[0.99525949] 0
[0.99423696] 0
[0.80024318] 0
[0.99911495] 0
[0.95505236] 0
[0.84463872] 0
[0.64235779] 0
[0.99758112] 0
[0.83094932] 0
[0.99932767] 0
[0.99074185] 0
[0.03078754] 1
[0.99894761] 0
[0.99427257] 0
[0.62165162] 0
[0.98589337] 0
[0.06246234] 1
[0.59149432] 0
[0.93050513] 0
[0.95589924] 0
[0.99871607] 0
[0.98307492] 0
[0.94632963] 0
[0.99999802] 0
[0.99733904] 0
[0.71331755] 0
[0.71723063] 0
[0.85590789] 0
[0.98816988] 0
[0.98176269] 0
[0.67168246] 0
[0.96818786] 

[0.97835121] 0
[0.99989302] 0
[0.95319293] 0
[0.67715839] 0
[0.99976279] 0
[0.78871219] 0
[0.95120312] 0
[0.99783469] 0
[0.78271695] 0
[0.99823385] 0
[0.9998457] 0
[0.99632368] 0
[0.5042155] 0
[0.5976458] 0
[0.7950068] 0
[7.70146366e-05] 1
[0.94684122] 0
[0.04501135] 1
[0.94250043] 0
[0.76526882] 0
[0.80592575] 0
[0.84300477] 0
[0.78567125] 0
[0.91981368] 0
[0.84425568] 0
[0.84173662] 0
[0.98662874] 0
[0.06617541] 1
[0.84488451] 0
[0.71217027] 0
[0.52791905] 0
[0.99961624] 0
[0.86673838] 0
[0.95936978] 0
[0.99935762] 0
[0.9998939] 0
[0.98515229] 0
[0.97640131] 0
[0.9969515] 0
[0.52392644] 0
[0.99984677] 0
[0.99824037] 0
[0.63440791] 0
[0.89618464] 0
[0.98953681] 0
[0.97738469] 0
[0.81314889] 0
[0.99945337] 0
[0.88810684] 0
[0.98475882] 0
[0.92957157] 0
[0.99999758] 0
[0.72403839] 0
[0.90780315] 0
[0.97086279] 0
[0.75720968] 0
[0.90000411] 0
[0.99690528] 0
[0.99426013] 0
[0.99548482] 0
[0.99705234] 0
[0.69633329] 0
[0.55975362] 0
[0.54653117] 0
[0.99854897] 0
[0.60474434] 0
[0.99277409]

[0.78706846] 0
[0.99532059] 0
[0.91732943] 0
[0.99990518] 0
[0.63637693] 0
[0.68144911] 0
[0.99999375] 0
[0.99922478] 0
[0.02591093] 1
[0.81473459] 0
[0.90646483] 0
[0.99931199] 0
[0.94311307] 0
[0.99318819] 0
[0.56655202] 0
[0.94149889] 0
[0.99633702] 0
[0.99968607] 0
[0.76819493] 0
[0.92359595] 0
[0.94091819] 0
[0.00120059] 1
[0.82011687] 0
[0.76618112] 0
[0.91842475] 0
[0.61812136] 0
[0.3977736] 1
[0.98803736] 0
[0.93096136] 0
[0.9723646] 0
[0.99959238] 0
[0.92974203] 0
[0.54463738] 0
[0.98345297] 0
[0.99797348] 0
[0.96707126] 0
[0.92727115] 0
[0.99462471] 0
[0.56723789] 0
[0.04826097] 1
[0.95492766] 0
[0.84606727] 0
[0.99999979] 0
[0.95762584] 0
[0.78903689] 0
[0.97957308] 0
[0.99937908] 0
[0.87669222] 0
[0.99515638] 0
[0.84464493] 0
[0.93237773] 0
[0.99849143] 0
[0.999851] 0
[0.70076028] 0
[0.98881439] 0
[0.98461707] 0
[0.10534177] 1
[0.73845484] 0
[0.83255617] 0
[0.99949393] 0
[0.96650512] 0
[0.9255309] 0
[0.87875675] 0
[0.86987655] 0
[0.63340767] 0
[0.17306595] 1
[0.99862875] 0
